In [3]:
# ======================================
# Task 1: Variational Autoencoder with CIFAR-10
# ======================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# ---------------------------
# Setup and CIFAR-10 Dataset
# ---------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

# ----------------------
# CNN-based VAE Modules
# ----------------------
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(128 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(128 * 4 * 4, latent_dim)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc_mu(x), self.fc_logvar(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(latent_dim, 128 * 4 * 4)
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1),
            nn.Sigmoid()
        )

    def forward(self, z):
        x = self.fc(z).view(z.size(0), 128, 4, 4)
        return self.deconv(x)

class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# ---------------
# Loss Function
# ---------------
def loss_function(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_div

# -------------------
# Training Function
# -------------------
def train_vae(latent_dim):
    model = VAE(latent_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    model.train()

    for epoch in range(50):
        total_loss = 0
        for x, _ in trainloader:
            x = x.to(device)
            optimizer.zero_grad()
            recon, mu, logvar = model(x)
            loss = loss_function(recon, x, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(trainloader.dataset):.4f}")
    return model

# -------------------------
# Experiment with Latent Dims
# -------------------------
latent_dims = [2, 8, 32, 128]
for dim in latent_dims:
    print(f"\nTraining with latent space dimension: {dim}")
    train_vae(latent_dim=dim)




Files already downloaded and verified

Training with latent space dimension: 2
Epoch 1, Loss: 131.7689
Epoch 2, Loss: 118.4501
Epoch 3, Loss: 118.0825
Epoch 4, Loss: 117.7829
Epoch 5, Loss: 117.6290
Epoch 6, Loss: 117.4452
Epoch 7, Loss: 117.2232
Epoch 8, Loss: 117.0634
Epoch 9, Loss: 116.9656
Epoch 10, Loss: 116.7814
Epoch 11, Loss: 116.8568
Epoch 12, Loss: 116.6505
Epoch 13, Loss: 116.5117
Epoch 14, Loss: 116.4588
Epoch 15, Loss: 116.4216
Epoch 16, Loss: 116.4101
Epoch 17, Loss: 116.3030
Epoch 18, Loss: 116.2121
Epoch 19, Loss: 116.2135
Epoch 20, Loss: 116.0058
Epoch 21, Loss: 115.9841
Epoch 22, Loss: 116.0242
Epoch 23, Loss: 115.9987
Epoch 24, Loss: 115.8109
Epoch 25, Loss: 115.8703
Epoch 26, Loss: 115.6153
Epoch 27, Loss: 115.7354
Epoch 28, Loss: 115.5573
Epoch 29, Loss: 115.4487
Epoch 30, Loss: 115.4214
Epoch 31, Loss: 115.2900
Epoch 32, Loss: 115.2835
Epoch 33, Loss: 115.3465
Epoch 34, Loss: 115.3220
Epoch 35, Loss: 115.0286
Epoch 36, Loss: 115.1271
Epoch 37, Loss: 115.0716
Epoch